In [1]:
import pandas as pd

import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context = 'notebook', #The base context is “notebook”, and the other contexts are “paper”, “talk”, and “poster”
        style = 'darkgrid', #dict, None, or one of {darkgrid, whitegrid, dark, white, ticks}
        palette = 'deep', # Should be something that color_palette() can process.
        font_scale = 1, 
        color_codes = False, 
        rc = None)

from IPython.display import display

%matplotlib notebook

np.version.version, pd.__version__

('1.14.2', '0.23.3')

In [2]:
# some functions to load

def head_with_full_columns(pd_in, row_amount = 5):
    with pd.option_context('display.max_columns', len(pd_in.iloc[0])):
        display(pd_in[:row_amount])
        
def balanced_sample(df_in, total_size, rand_state):
    s0 = df_in[df_in['TARGET']==0].sample(n = total_size//2, random_state = rand_state)
    s1 = df_in[df_in['TARGET']==1].sample(n = total_size//2, random_state = rand_state)
    new_df = pd.concat([s0,s1])
    new_df.sort_index(inplace = True)
    return new_df

def which_df(feature):
    if feature in application_train_df_f_list:
        return feature, 'application_train_df'
    
    if feature in bureau_df_f_list:
        return feature, 'bureau_df'
        
    if feature in bureau_balance_df_f_list:
        return feature, 'bureau_balance_df'
        
    if feature in credit_card_balance_df_f_list:
        return feature, 'credit_card_balance_df'
    
    if feature in installments_payments_df_f_list:
        return feature, 'installments_payments_df'
        
    if feature in POS_CASH_balance_df_f_list:
        return feature, 'POS_CASH_balance_df'
        
    if feature in previous_application_df_f_list:
        return feature, 'previous_application_df'

In [2]:
pwd

'/Users/DonBunk/Desktop/Google Drive/data_science/Python_Projects/Home_Credit_Default_Risk/aggregation/TEST_aggregation'

In [4]:
cd '/Users/DonBunk/Desktop/Google Drive/data_science/Python_Projects/Home_Credit_Default_Risk/'

/Users/DonBunk/Desktop/Google Drive/data_science/Python_Projects/Home_Credit_Default_Risk


In [5]:
path_w = 'raw_loan_data_from_Kaggle/'
path_a = 'aggregation/TEST_aggregation/'

In [6]:
test_SK_ID_index_df = pd.read_csv(path_w + 'application_test.csv', usecols=['SK_ID_CURR'], index_col = 'SK_ID_CURR')

test_SK_ID_index_df.sort_values(by = ['SK_ID_CURR'], inplace=True)

# aggregate data for each df and merge dfs

## load bureau csv

In [8]:
bureau_df = pd.read_csv(path_w + 'bureau.csv')#, index_col = 'SK_ID_CURR')
# replace NaNs in object entries for now is pretty safe. 
f_list = list(bureau_df.select_dtypes('object').columns)
bureau_df[f_list] = bureau_df[f_list].fillna(value = 'NA')
# AMT_ANNUITY appears in multiple files, so rename more specifically. 
bureau_df.rename( columns = {"AMT_ANNUITY":"AMT_ANNUITY_from_bureau"} , inplace=True)# index = str,
del f_list

### aggregate and define df

In [9]:
bureau_df.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1716428 entries, 0 to 1716427
Data columns (total 17 columns):
SK_ID_CURR                 1716428 non-null int64
SK_ID_BUREAU               1716428 non-null int64
CREDIT_ACTIVE              1716428 non-null object
CREDIT_CURRENCY            1716428 non-null object
DAYS_CREDIT                1716428 non-null int64
CREDIT_DAY_OVERDUE         1716428 non-null int64
DAYS_CREDIT_ENDDATE        1610875 non-null float64
DAYS_ENDDATE_FACT          1082775 non-null float64
AMT_CREDIT_MAX_OVERDUE     591940 non-null float64
CNT_CREDIT_PROLONG         1716428 non-null int64
AMT_CREDIT_SUM             1716415 non-null float64
AMT_CREDIT_SUM_DEBT        1458759 non-null float64
AMT_CREDIT_SUM_LIMIT       1124648 non-null float64
AMT_CREDIT_SUM_OVERDUE     1716428 non-null float64
CREDIT_TYPE                1716428 non-null object
DAYS_CREDIT_UPDATE         1716428 non-null int64
AMT_ANNUITY_from_bureau    489637 non-null float64
dtypes: float64(8), 

In [10]:
len(np.unique(bureau_df['SK_ID_CURR'].values))

305811

All the floats are really floats, and all the integers are continous numerical.

In [11]:
# group interger features
bureau_df_float_list = list(bureau_df.select_dtypes('float64').columns)
bureau_df_float_grouped = bureau_df[bureau_df_float_list].groupby(bureau_df['SK_ID_CURR'])

# apply functions to these features and name them appropriately
function_list = ['mean', 'median','max','min'] 
bureau_df_float_grouped_agg = bureau_df_float_grouped.agg(function_list)
new_cols = ['_'.join(col).strip() for col in bureau_df_float_grouped_agg.columns.values]
bureau_df_float_grouped_agg.columns = new_cols

In [12]:
# group float features, remove ID features
bureau_df_int_list = list(    bureau_df.select_dtypes('int64').columns    )
bureau_df_int_list.remove('SK_ID_CURR')
bureau_df_int_list.remove('SK_ID_BUREAU')
bureau_df_int_grouped = bureau_df[bureau_df_int_list].groupby(bureau_df['SK_ID_CURR'])

# apply functions to these features and name them appropriately
function_list = ['mean', 'median','max','min']
bureau_df_int_grouped_agg = bureau_df_int_grouped.agg(function_list)
new_cols = ['_'.join(col).strip() for col in bureau_df_int_grouped_agg.columns.values]
bureau_df_int_grouped_agg.columns = new_cols

In [13]:
# group string features
bureau_df_string_list = list(bureau_df.select_dtypes('object').columns)
bureau_df_string_grouped = bureau_df[bureau_df_string_list].groupby(bureau_df['SK_ID_CURR'])

# calculate the mode for each aggregated feature
# this takes a VERY long time to run
bureau_df_string_grouped_agg = bureau_df_string_grouped.agg(lambda x:x.value_counts().index[0])
bureau_df_string_grouped_agg.columns = [x + '_mode' for x in bureau_df_string_grouped_agg.columns]

In [16]:
# merge int, float, and string features into new df
bureau_df_float_and_int_grouped = pd.merge(bureau_df_float_grouped_agg, bureau_df_int_grouped_agg, left_index=True, right_index=True)
bureau_df_aggregated_final = pd.merge(bureau_df_float_and_int_grouped, bureau_df_string_grouped_agg, left_index=True, right_index=True)

In [18]:
bureau_df_aggregated_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 305811 entries, 100001 to 456255
Data columns (total 51 columns):
DAYS_CREDIT_ENDDATE_mean          303226 non-null float64
DAYS_CREDIT_ENDDATE_median        303226 non-null float64
DAYS_CREDIT_ENDDATE_max           303226 non-null float64
DAYS_CREDIT_ENDDATE_min           303226 non-null float64
DAYS_ENDDATE_FACT_mean            268155 non-null float64
DAYS_ENDDATE_FACT_median          268155 non-null float64
DAYS_ENDDATE_FACT_max             268155 non-null float64
DAYS_ENDDATE_FACT_min             268155 non-null float64
AMT_CREDIT_MAX_OVERDUE_mean       212971 non-null float64
AMT_CREDIT_MAX_OVERDUE_median     212971 non-null float64
AMT_CREDIT_MAX_OVERDUE_max        212971 non-null float64
AMT_CREDIT_MAX_OVERDUE_min        212971 non-null float64
AMT_CREDIT_SUM_mean               305809 non-null float64
AMT_CREDIT_SUM_median             305809 non-null float64
AMT_CREDIT_SUM_max                305809 non-null float64
AMT_CREDIT_SUM

In [19]:
# add in all SK_IDS, which will appear as NaNs in new df
bureau_df_aggregated_final = pd.merge(test_SK_ID_index_df, bureau_df_aggregated_final, left_index=True, right_index=True, how = 'left')

In [20]:
bureau_df_aggregated_final.to_csv(path_a + 'bureau_df_aggregated_final.csv', columns = list(bureau_df_aggregated_final.columns))

In [21]:
del bureau_df
del bureau_df_float_list
del bureau_df_float_grouped
del bureau_df_float_grouped_agg
del bureau_df_int_list
del bureau_df_int_grouped
del function_list
del bureau_df_int_grouped_agg 
del bureau_df_string_list 
del bureau_df_string_grouped
del bureau_df_string_grouped_agg 
del bureau_df_float_and_int_grouped
del new_cols 

## installments_payments_df

In [22]:
installments_payments_df = pd.read_csv(path_w + 'installments_payments.csv')#, index_col = 'SK_ID_CURR')
# replace NaNs in object entries for now is pretty safe. 
f_list = list(installments_payments_df.select_dtypes('object').columns)
installments_payments_df[f_list] = installments_payments_df[f_list].fillna(value = 'NA')
del f_list

### add new column features to full df

In [24]:
installments_payments_df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13605401 entries, 0 to 13605400
Data columns (total 8 columns):
SK_ID_PREV                13605401 non-null int64
SK_ID_CURR                13605401 non-null int64
NUM_INSTALMENT_VERSION    13605401 non-null float64
NUM_INSTALMENT_NUMBER     13605401 non-null int64
DAYS_INSTALMENT           13605401 non-null float64
DAYS_ENTRY_PAYMENT        13602496 non-null float64
AMT_INSTALMENT            13605401 non-null float64
AMT_PAYMENT               13602496 non-null float64
dtypes: float64(5), int64(3)
memory usage: 830.4 MB


In [25]:
# add these new features in full DF. 
installments_payments_df['DAYS_PAYMENT_LATE'] = installments_payments_df['DAYS_ENTRY_PAYMENT'] - installments_payments_df['DAYS_INSTALMENT']
installments_payments_df['AMT_OVERPAY'] = installments_payments_df['AMT_PAYMENT'] - installments_payments_df['AMT_INSTALMENT']

In [26]:
# organize indices to call later. 
installments_payments_df.set_index(['SK_ID_CURR','SK_ID_PREV', 'NUM_INSTALMENT_NUMBER'], inplace = True)
installments_payments_df.sort_index(inplace=True)

In [27]:
# this takes WHILE to run for the full df. 

# peel off, group, and find diffs of NUM_INSTALMENT_VERSION to see how often loan payment calender changed. 

# NOTE YOU NEED DOUBLE SQUARE BRACKETS BELOW SO THAT YOU DEFINE A DF NOT A SERIES, group by and diff only seem to work on former!
NUM_INSTALMENT_VERSION_df = installments_payments_df[['NUM_INSTALMENT_VERSION']]
NUM_INSTALMENT_VERSION_df_grouped = NUM_INSTALMENT_VERSION_df[['NUM_INSTALMENT_VERSION']].groupby(level = 1)

# THIS TAKES AWHILE!
NUM_INSTALMENT_VERSION_df_grouped_diff = NUM_INSTALMENT_VERSION_df_grouped.diff()

In [28]:
# rename this new feature and join with original full DF. 
NUM_INSTALMENT_VERSION_df_grouped_diff['NUM_INSTALMENT_VERSION_diff'] = NUM_INSTALMENT_VERSION_df_grouped_diff['NUM_INSTALMENT_VERSION']
installments_payments_df = installments_payments_df.join( NUM_INSTALMENT_VERSION_df_grouped_diff['NUM_INSTALMENT_VERSION_diff'])

In [30]:
# reset NUM_INSTALMENT_NUMBER and SK_ID_PREV to columns
installments_payments_df.reset_index(level=2, inplace=True)
installments_payments_df.reset_index(level=1, inplace=True)

In [32]:
def count_unique(list_in):
    return len(np.unique(list_in))

def count_postive(list_in):
    return len([x for x in list_in if x > 0])

def count_negative(list_in):
    return len([x for x in list_in if x < 0])

In [33]:
installments_payments_df_grouped = installments_payments_df.groupby('SK_ID_CURR')

# create the new feature and make a new summary feature DF out of it. 
num_of_loans = installments_payments_df_grouped['SK_ID_PREV'].agg([('NUM_OF_LOANS', count_unique)])
installments_payments_df_summary = num_of_loans

In [34]:
# demote SK_ID_CURR index to column for aggregation
installments_payments_df.reset_index(level=0, inplace=True)

In [35]:
# group for aggregation
installments_payments_df_grouped = installments_payments_df.groupby('SK_ID_CURR')

In [36]:
# early/ late features. 

agg_list = [('DAYS_PAYMENT_LATE_mean', np.mean), 
            ('DAYS_PAYMENT_LATE_median', np.median),
            ('DAYS_PAYMENT_LATE_sd', np.std),
            ('NUM_TIMES_LATE', count_postive),
            ('NUM_TIMES_EARLY', count_negative)]

some_feats = installments_payments_df_grouped['DAYS_PAYMENT_LATE'].agg(agg_list)

installments_payments_df_summary = installments_payments_df_summary.join(some_feats)

In [37]:
# over/under pay features. 

agg_list = [('AMT_OVERPAY_MEAN', np.mean), 
            ('AMT_OVERPAY_MEDIAN', np.median),
            ('AMT_OVERPAY_SD', np.std),
            ('NUM_TIMES_OVERPAY', count_postive),
            ('NUM_TIMES_UNDERPAY', count_negative)]

some_feats = installments_payments_df_grouped['NUM_INSTALMENT_VERSION_diff'].agg(agg_list)

installments_payments_df_summary = installments_payments_df_summary.join(some_feats)

In [38]:
# loan terms change features. 

agg_list = [('TERMS_CHANGE_TIMES', np.count_nonzero)]

some_feats = installments_payments_df_grouped['NUM_INSTALMENT_VERSION_diff'].agg(agg_list)

installments_payments_df_summary = installments_payments_df_summary.join(some_feats)

In [39]:
installments_payments_df_summary['TERMS_CHANGE_PER_LOAN'] \
= installments_payments_df_summary['TERMS_CHANGE_TIMES']/installments_payments_df_summary['NUM_OF_LOANS']

In [41]:
installments_payments_df_summary.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 339587 entries, 100001 to 456255
Data columns (total 13 columns):
NUM_OF_LOANS                339587 non-null int64
DAYS_PAYMENT_LATE_mean      339578 non-null float64
DAYS_PAYMENT_LATE_median    339578 non-null float64
DAYS_PAYMENT_LATE_sd        338610 non-null float64
NUM_TIMES_LATE              339587 non-null float64
NUM_TIMES_EARLY             339587 non-null float64
AMT_OVERPAY_MEAN            338569 non-null float64
AMT_OVERPAY_MEDIAN          338569 non-null float64
AMT_OVERPAY_SD              337656 non-null float64
NUM_TIMES_OVERPAY           339587 non-null float64
NUM_TIMES_UNDERPAY          339587 non-null float64
TERMS_CHANGE_TIMES          339587 non-null float64
TERMS_CHANGE_PER_LOAN       339587 non-null float64
dtypes: float64(12), int64(1)
memory usage: 46.3 MB


In [43]:
del NUM_INSTALMENT_VERSION_df
del NUM_INSTALMENT_VERSION_df_grouped
del NUM_INSTALMENT_VERSION_df_grouped_diff 
del num_of_loans 
del installments_payments_df_grouped
del agg_list 
del some_feats

### basic aggregate data

In [44]:
# have to drop these columns if I have previously ran the section above. 
installments_payments_df.drop(columns = ['DAYS_PAYMENT_LATE','AMT_OVERPAY','NUM_INSTALMENT_VERSION_diff'], inplace = True)

In [46]:
installments_payments_df.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15120797 entries, 0 to 15120796
Data columns (total 8 columns):
SK_ID_CURR                15120797 non-null int64
SK_ID_PREV                15120797 non-null int64
NUM_INSTALMENT_NUMBER     15120797 non-null int64
NUM_INSTALMENT_VERSION    15120797 non-null float64
DAYS_INSTALMENT           15120797 non-null float64
DAYS_ENTRY_PAYMENT        15117863 non-null float64
AMT_INSTALMENT            15120797 non-null float64
AMT_PAYMENT               15117863 non-null float64
dtypes: float64(5), int64(3)
memory usage: 922.9 MB


All the floats are really floats, and all the integers are continous numerical.
No string features. 

In [50]:
installments_payments_df_float_grouped = installments_payments_df.groupby(installments_payments_df['SK_ID_CURR'])

In [51]:
# group the float features
installments_payments_df_float_list = list(installments_payments_df.select_dtypes('float64').columns)

installments_payments_df_float_grouped = \
installments_payments_df[installments_payments_df_float_list].groupby(installments_payments_df['SK_ID_CURR'])

# apply these functions to the aggregated feaures, and name acoordingly
function_list = ['mean', 'median', 'max', 'min']
installments_payments_df_float_grouped_agg = installments_payments_df_float_grouped.agg(function_list)

new_cols = ['_'.join(col).strip() for col in installments_payments_df_float_grouped_agg.columns.values]
installments_payments_df_float_grouped_agg.columns = new_cols

In [ ]:
# group the int features
installments_payments_df_int_list = list(    installments_payments_df.select_dtypes('int64').columns    )
installments_payments_df_int_list.remove('SK_ID_CURR')
installments_payments_df_int_list.remove('SK_ID_PREV')

installments_payments_df_int_grouped = \
installments_payments_df[installments_payments_df_int_list].groupby(installments_payments_df['SK_ID_CURR'])

# apply these functions to the aggregated feaures, and name acoordingly
function_list = ['mean', 'median','max','min']
installments_payments_df_int_grouped_agg = installments_payments_df_int_grouped.agg(function_list)

new_cols = ['_'.join(col).strip() for col in installments_payments_df_int_grouped_agg.columns.values]
installments_payments_df_int_grouped_agg.columns = new_cols

In [ ]:
# merge all the above as new df
installments_payments_df_aggregated_final = \
pd.merge(installments_payments_df_float_grouped_agg, installments_payments_df_int_grouped_agg, left_index=True, right_index=True)

In [54]:
del installments_payments_df_float_list
del installments_payments_df_float_grouped 
del function_list
del installments_payments_df_float_grouped_agg
del new_cols 
del installments_payments_df_int_list 
del installments_payments_df_int_grouped
del installments_payments_df_int_grouped_agg

### merge dfs from two sections above

In [55]:
# add in all SK_IDS, which will appear as NaNs in new df
installments_payments_df_final = pd.merge(test_SK_ID_index_df, installments_payments_df_summary, left_index=True, right_index=True, how = 'left')
installments_payments_df_final = pd.merge(installments_payments_df_final, installments_payments_df_aggregated_final, left_index=True, right_index=True, how = 'left')

In [56]:
installments_payments_df_final.to_csv(path_a + 'installments_payments_df_final.csv', columns = list(installments_payments_df_final.columns))

In [57]:
installments_payments_df_final.columns

Index(['NUM_OF_LOANS', 'DAYS_PAYMENT_LATE_mean', 'DAYS_PAYMENT_LATE_median',
       'DAYS_PAYMENT_LATE_sd', 'NUM_TIMES_LATE', 'NUM_TIMES_EARLY',
       'AMT_OVERPAY_MEAN', 'AMT_OVERPAY_MEDIAN', 'AMT_OVERPAY_SD',
       'NUM_TIMES_OVERPAY', 'NUM_TIMES_UNDERPAY', 'TERMS_CHANGE_TIMES',
       'TERMS_CHANGE_PER_LOAN', 'NUM_INSTALMENT_VERSION_mean',
       'NUM_INSTALMENT_VERSION_median', 'NUM_INSTALMENT_VERSION_max',
       'NUM_INSTALMENT_VERSION_min', 'DAYS_INSTALMENT_mean',
       'DAYS_INSTALMENT_median', 'DAYS_INSTALMENT_max', 'DAYS_INSTALMENT_min',
       'DAYS_ENTRY_PAYMENT_mean', 'DAYS_ENTRY_PAYMENT_median',
       'DAYS_ENTRY_PAYMENT_max', 'DAYS_ENTRY_PAYMENT_min',
       'AMT_INSTALMENT_mean', 'AMT_INSTALMENT_median', 'AMT_INSTALMENT_max',
       'AMT_INSTALMENT_min', 'AMT_PAYMENT_mean', 'AMT_PAYMENT_median',
       'AMT_PAYMENT_max', 'AMT_PAYMENT_min', 'NUM_INSTALMENT_NUMBER_mean',
       'NUM_INSTALMENT_NUMBER_median', 'NUM_INSTALMENT_NUMBER_max',
       'NUM_INSTALMENT_NUMBER_

In [58]:
del installments_payments_df_summary
del installments_payments_df
del installments_payments_df_final
del installments_payments_df_aggregated_final

##  POS_CASH_balance_df

In [59]:
POS_CASH_balance_df = pd.read_csv(path_w + 'POS_CASH_balance.csv')
# replace NaNs in object entries for now is pretty safe. 

f_list = list(POS_CASH_balance_df.select_dtypes('object').columns)
POS_CASH_balance_df[f_list] = POS_CASH_balance_df[f_list].fillna(value = 'NA')
del f_list

### Aggregate data

In [61]:
POS_CASH_balance_df.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001358 entries, 0 to 10001357
Data columns (total 8 columns):
SK_ID_PREV               10001358 non-null int64
SK_ID_CURR               10001358 non-null int64
MONTHS_BALANCE           10001358 non-null int64
CNT_INSTALMENT           9975287 non-null float64
CNT_INSTALMENT_FUTURE    9975271 non-null float64
NAME_CONTRACT_STATUS     10001358 non-null object
SK_DPD                   10001358 non-null int64
SK_DPD_DEF               10001358 non-null int64
dtypes: float64(2), int64(5), object(1)
memory usage: 610.4+ MB


All the floats are really floats, and all the integers are continous numerical.

In [64]:
# group all the float features and aggregate
POS_CASH_balance_df_float_list = list(POS_CASH_balance_df.select_dtypes('float64').columns)

POS_CASH_balance_df_float_grouped = POS_CASH_balance_df[POS_CASH_balance_df_float_list].groupby(POS_CASH_balance_df['SK_ID_CURR'])

# apply the functions to the aggregated features and name accordingly
function_list = ['mean', 'median','max','min']
POS_CASH_balance_df_float_grouped_agg = POS_CASH_balance_df_float_grouped.agg(function_list)
new_cols = ['_'.join(col).strip() for col in POS_CASH_balance_df_float_grouped_agg.columns.values]
POS_CASH_balance_df_float_grouped_agg.columns = new_cols

In [65]:
# group all int features and aggregate, drop ID features
POS_CASH_balance_df_int_list = list(    POS_CASH_balance_df.select_dtypes('int64').columns    )
POS_CASH_balance_df_int_list.remove('SK_ID_CURR')
POS_CASH_balance_df_int_list.remove('SK_ID_PREV')

POS_CASH_balance_df_int_grouped = POS_CASH_balance_df[POS_CASH_balance_df_int_list].groupby(POS_CASH_balance_df['SK_ID_CURR'])

# apply the functions to the aggregated features and name accordingly
function_list = ['mean', 'median','max','min']
POS_CASH_balance_df_int_grouped_agg = POS_CASH_balance_df_int_grouped.agg(function_list)
new_cols = ['_'.join(col).strip() for col in POS_CASH_balance_df_int_grouped_agg.columns.values]
POS_CASH_balance_df_int_grouped_agg.columns = new_cols

In [66]:
# group all object features and aggregate
POS_CASH_balance_df_string_list = list(POS_CASH_balance_df.select_dtypes('object').columns)
POS_CASH_balance_df_string_grouped = POS_CASH_balance_df[POS_CASH_balance_df_string_list].groupby(POS_CASH_balance_df['SK_ID_CURR'])

# calculate the mode for these features and name accordingly
# this takes a long time to run
POS_CASH_balance_df_string_grouped_agg = POS_CASH_balance_df_string_grouped.agg(lambda x:x.value_counts().index[0])

POS_CASH_balance_df_string_grouped_agg.columns = [x + '_mode' for x in POS_CASH_balance_df_string_grouped_agg.columns]

In [69]:
# merge above dfs
POS_CASH_balance_df_float_and_int_grouped = pd.merge(POS_CASH_balance_df_float_grouped_agg, POS_CASH_balance_df_int_grouped_agg, left_index=True, right_index=True)
POS_CASH_balance_df_aggregated_final = pd.merge(POS_CASH_balance_df_float_and_int_grouped, POS_CASH_balance_df_string_grouped_agg, left_index=True, right_index=True)

In [74]:
# add in all SK_IDS, which will appear as NaNs in new df
POS_CASH_balance_df_aggregated_final = pd.merge(test_SK_ID_index_df, POS_CASH_balance_df_aggregated_final, left_index=True, right_index=True, how = 'left')

In [75]:
POS_CASH_balance_df_aggregated_final.to_csv(path_a +'POS_CASH_balance_df_aggregated_final.csv', columns = list(POS_CASH_balance_df_aggregated_final.columns))

In [76]:
del POS_CASH_balance_df
del POS_CASH_balance_df_aggregated_final
del POS_CASH_balance_df_float_list 
del POS_CASH_balance_df_float_grouped 
del function_list
del POS_CASH_balance_df_float_grouped_agg
del new_cols 
del POS_CASH_balance_df_int_list 
del POS_CASH_balance_df_int_grouped 
del POS_CASH_balance_df_int_grouped_agg
del POS_CASH_balance_df_string_list
del POS_CASH_balance_df_string_grouped
del POS_CASH_balance_df_string_grouped_agg 
del POS_CASH_balance_df_float_and_int_grouped

##  previous_application_df

In [77]:
previous_application_df = pd.read_csv(path_w + 'previous_application.csv')
# replace NaNs in object entries for now is pretty safe. 

f_list = list(previous_application_df.select_dtypes('object').columns)
previous_application_df[f_list] = previous_application_df[f_list].fillna(value = 'NA')
del f_list

### Aggregate data

In [78]:
# I have to rename this bc there is an AMT_ANNUITY and NAME_CONTRACT_STATUS in the application data set too, and need to distinguish after merging. 
previous_application_df.rename(columns= {'AMT_ANNUITY':'AMT_ANNUITY_PREV_APP', 'NAME_CONTRACT_STATUS':'NAME_CONTRACT_STATUS_PREV_APP'}, inplace=True)

In [80]:
previous_application_df.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1670214 entries, 0 to 1670213
Data columns (total 37 columns):
SK_ID_PREV                       1670214 non-null int64
SK_ID_CURR                       1670214 non-null int64
NAME_CONTRACT_TYPE               1670214 non-null object
AMT_ANNUITY_PREV_APP             1297979 non-null float64
AMT_APPLICATION                  1670214 non-null float64
AMT_CREDIT                       1670213 non-null float64
AMT_DOWN_PAYMENT                 774370 non-null float64
AMT_GOODS_PRICE                  1284699 non-null float64
WEEKDAY_APPR_PROCESS_START       1670214 non-null object
HOUR_APPR_PROCESS_START          1670214 non-null int64
FLAG_LAST_APPL_PER_CONTRACT      1670214 non-null object
NFLAG_LAST_APPL_IN_DAY           1670214 non-null int64
RATE_DOWN_PAYMENT                774370 non-null float64
RATE_INTEREST_PRIMARY            5951 non-null float64
RATE_INTEREST_PRIVILEGED         5951 non-null float64
NAME_CASH_LOAN_PURPOSE           167

All the floats are really floats, and all the integers are continous numerical except NFLAG_LAST_APPL_IN_DAY so subtract that from int,
and add to catagorical list. 

In [84]:
# get float features and group them 
previous_application_df_float_list = list(previous_application_df.select_dtypes('float64').columns)
previous_application_df_float_grouped = previous_application_df[previous_application_df_float_list].groupby(previous_application_df['SK_ID_CURR'])

# apply these functions and name new features accordinly
function_list = ['mean', 'median','max','min']
previous_application_df_float_grouped_agg = previous_application_df_float_grouped.agg(function_list)
new_cols = ['_'.join(col).strip() for col in previous_application_df_float_grouped_agg.columns.values]
previous_application_df_float_grouped_agg.columns = new_cols

In [85]:
# get int features and group them, and drop ID features
previous_application_df_int_list = list(    previous_application_df.select_dtypes('int64').columns    )
previous_application_df_int_list.remove('SK_ID_CURR')
previous_application_df_int_list.remove('SK_ID_PREV')
previous_application_df_int_list.remove('NFLAG_LAST_APPL_IN_DAY')

previous_application_df_int_grouped = previous_application_df[previous_application_df_int_list].groupby(previous_application_df['SK_ID_CURR'])

# apply these functions and name new features accordinly
function_list = ['mean', 'median','max','min']
previous_application_df_int_grouped_agg = previous_application_df_int_grouped.agg(function_list)
new_cols = ['_'.join(col).strip() for col in previous_application_df_int_grouped_agg.columns.values]
previous_application_df_int_grouped_agg.columns = new_cols

In [86]:
previous_application_df_string_list = list(previous_application_df.select_dtypes('object').columns) + ['NFLAG_LAST_APPL_IN_DAY']

previous_application_df_string_grouped \
= previous_application_df[previous_application_df_string_list].groupby(previous_application_df['SK_ID_CURR'])

# calculate the mode for these features and name accordingly
# this takes a long time to run
previous_application_df_string_grouped_agg \
= previous_application_df_string_grouped.agg(lambda x:x.value_counts().index[0])
previous_application_df_string_grouped_agg;

previous_application_df_string_grouped_agg.columns = [x + '_mode' for x in previous_application_df_string_grouped_agg.columns]

In [91]:
# merge all these dfs together
previous_application_df_float_and_int_grouped = \
pd.merge(previous_application_df_float_grouped_agg, previous_application_df_int_grouped_agg, left_index=True, right_index=True)
previous_application_df_aggregated_final = \
pd.merge(previous_application_df_float_and_int_grouped, previous_application_df_string_grouped_agg, left_index=True, right_index=True)

In [95]:
# add in all SK_IDS, which will appear as NaNs in new df

previous_application_df_aggregated_final = pd.merge(test_SK_ID_index_df, previous_application_df_aggregated_final, left_index=True, right_index=True, how = 'left')

In [96]:
previous_application_df_aggregated_final.to_csv(path_a + 'previous_application_df_aggregated_final.csv', columns = list(previous_application_df_aggregated_final.columns))

In [97]:
del previous_application_df
del previous_application_df_aggregated_final
del previous_application_df_float_list
del previous_application_df_float_grouped 
del function_list
del previous_application_df_float_grouped_agg
del new_cols 
del previous_application_df_int_list 
del previous_application_df_int_grouped
del previous_application_df_string_list
del previous_application_df_string_grouped
del previous_application_df_string_grouped_agg
del previous_application_df_float_and_int_grouped